In [83]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import requests
import streamlit as st

In [84]:
# Load dataset
df = pd.read_csv("updated_weather_data.csv")
df

,Date,City,Temperature,Humidity,Pressure,Wind Speed
0,2022-01-01,Mumbai,38.64,52,1014,5.71
1,2022-01-02,Delhi,26.28,78,1011,6.73
2,2022-01-03,Bangalore,15.58,82,1004,3.46
3,2022-01-04,Chennai,30.52,72,1007,7.67
4,2022-01-05,Kolkata,27.20,84,1019,8.20
...,...,...,...,...,...,...
4995,2024-09-22,Mumbai,35.58,90,1023,7.59
4996,2024-09-23,Delhi,15.67,65,1006,9.96
4997,2024-09-24,Bangalore,33.36,78,1007,5.89
4998,2024-09-25,Chennai,30.08,66,1015,7.84


In [85]:
# Extract city names
cities = df['City'].unique()


In [86]:
# Data Preparation
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append((seq, label))
    return sequences

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Temperature'].values.reshape(-1, 1))

seq_length = 10  # Number of time steps to consider in LSTM
dataset = create_sequences(scaled_data, seq_length)

train_size = int(len(dataset) * 0.8)
train_data = dataset[:train_size]
test_data = dataset[train_size:]

def convert_to_tensor(data):
    X = torch.tensor([seq for seq, _ in data], dtype=torch.float32)
    y = torch.tensor([label for _, label in data], dtype=torch.float32)
    return X, y

X_train, y_train = convert_to_tensor(train_data)
X_test, y_test = convert_to_tensor(test_data)


In [87]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=10, hidden_layer_size=50, output_size=1):  # Change input_size to 10
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]


In [88]:
# Initialize model, loss function, and optimizer
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [89]:
# Training Loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                         torch.zeros(1, 1, model.hidden_layer_size))

    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {loss.item():.4f}')



C:\Users\Krishna\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([3992, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 5/50 - Loss: 0.2678
Epoch 10/50 - Loss: 0.1517
Epoch 15/50 - Loss: 0.0873
Epoch 20/50 - Loss: 0.0976
Epoch 25/50 - Loss: 0.0927
Epoch 30/50 - Loss: 0.0835
Epoch 35/50 - Loss: 0.0854
Epoch 40/50 - Loss: 0.0859
Epoch 45/50 - Loss: 0.0841
Epoch 50/50 - Loss: 0.0835


In [90]:
# Save the trained model
torch.save(model.state_dict(), r"C:\Users\Krishna\Desktop\Weather\model.pth")
print("Model trained and saved successfully!")

Model trained and saved successfully!


In [94]:
# Load the model
model = LSTMModel()
model.load_state_dict(torch.load(r"C:\Users\Krishna\Desktop\Weather\model.pth"))
model.eval()
print("Model loaded successfully!")


Model loaded successfully!


In [96]:
# API Configuration
api_key = "358d4712052db8663c857eec875c24d6"
base_url = "http://api.openweathermap.org/data/2.5/weather"


In [98]:
def predict_temperatures(city):
    response = requests.get(f"{base_url}?q={city}&appid={api_key}&units=metric")
    data = response.json()

    if response.status_code == 200:
        current_temp = data['main']['temp']

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_temp = scaler.fit_transform(np.array([current_temp]).reshape(-1, 1))
        temp_tensor = torch.tensor(scaled_temp, dtype=torch.float32)

        predictions = []
        for _ in range(3):
            with torch.no_grad():
                predicted_temp = model(temp_tensor).item()
            temp_tensor = torch.cat((temp_tensor, torch.tensor([[predicted_temp]])))
            predictions.append(predicted_temp)

        predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
        return f"Predicted temperatures for next 3 days in {city}: {predictions.flatten()}°C"
    else:
        return f"Error fetching data for {city}."


In [102]:
# Streamlit UI
st.title("Weather Prediction App")
selected_city = st.selectbox("Select a city:", cities)
if st.button("Predict Temperature"):
    result = predict_temperatures(selected_city)
    st.write(result)

2025-03-20 11:13:55.369 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 11:13:55.456 
  command:

    streamlit run C:\Users\Krishna\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-20 11:13:55.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 11:13:55.457 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 11:13:55.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 11:13:55.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 11:13:55.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 11:13:55.463 Session state does no